In [145]:
import pandas as pd
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [103]:
# bring in the 311 data

data311_df = pd.read_csv("../Clean Data Files/modelstartdata.csv")


print(data311_df.columns.values)
# print(data311_df["sr create date"])
data311_df.head()

['index' 'case number' 'sr location' 'county' 'district' 'neighborhood'
 'tax id' 'trash quad' 'recycle quad' 'trash day' 'heavy trash day'
 'recycle day' 'key map' 'management district' 'department' 'division'
 'sr type' 'queue' 'sla' 'status' 'sr create date' 'due date'
 'date closed' 'overdue' 'title' 'x' 'y' 'latitude' 'longitude'
 'channel type' 'zipcode' 'openclosetime' 'openduetime' 'year']


,index,case number,sr location,county,district,neighborhood,tax id,trash quad,recycle quad,trash day,...,title,x,y,latitude,longitude,channel type,zipcode,openclosetime,openduetime,year
0,6,12091836-101002444730,"3303 SAGE, HOUSTON TX 77056",HARRIS,G,GREATER UPTOWN,4.514000e+11,NaN,NaN,NaN,...,Water Leak-101002444730,3089853.524,13831195.74,29.733522,-95.466104,Voice In,77056,32 days 12:49:18.000000000,9 days 23:59:59.000000000,2017
1,12,12091839-101002444736,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,7.102100e+11,NE,NW,MONDAY,...,Sewer Wastewater-101002444736,3119884.277,13867666.56,29.831239,-95.367955,Voice In,77022,3 days 05:37:27.000000000,0 days 23:59:58.000000000,2017
2,13,12091840-101002444737,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,4.200500e+11,NE,NW,MONDAY,...,Water Service-101002444737,3119785.781,13867661.60,29.831234,-95.368266,Voice In,77022,2 days 22:16:31.000000000,2 days 00:00:00.000000000,2017
3,15,12091841-101002444739,"7612 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,7.102100e+11,NE,NW,MONDAY,...,Water Service-101002444739,3120069.146,13867375.14,29.830423,-95.367401,Voice In,77022,0 days 02:26:29.000000000,1 days 23:59:59.000000000,2017
4,23,12091843-101002444752,"7335 IWO JIMA, HOUSTON TX 77033",HARRIS,D,SOUTH PARK,7.518900e+11,SE,SE,FRIDAY,...,Water Leak-101002444752,3128978.891,13812277.81,29.678215,-95.344744,Voice In,77033,0 days 02:34:10.000000000,10 days 00:00:00.000000000,2017


In [159]:
################################################################
# create a date value formatted the same as the weather data

splitdate = data311_df["sr create date"].str.split(" ", n=1, expand=True)

data311_df["create_date"] = splitdate[0]
data311_df["create_date"] = pd.to_datetime(data311_df["create_date"])
data311_df["create_date"] = data311_df['create_date'].dt.strftime('%Y-%m-%d')

##################################################################
# below is where I create the variables that I can use for modeling. I originally was trying to use the openclose time
# columns, but couldn't get it to just show the number of days.

data311.head()

type(data311_df["sr create date"])

# I set these as date formats using the to_datetime

data311_df["create_date_time"] = pd.to_datetime(data311_df["sr create date"])
data311_df["date_closed_time"] = pd.to_datetime(data311_df["date closed"])
data311_df["due_date_time"] = pd.to_datetime(data311_df["due date"])


# here  I define weather is was closed by the due date (0) or not (1)
data311_df["missed_due_date"] = np.where(data311_df["due_date_time"] > data311_df["date_closed_time"], 0, 1)

# calculate the difference between the closed date at the create date
data311_df["time_to_close"] = data311_df["date_closed_time"] - data311_df["create_date_time"]

# apply formatting so that is is in seconds. then calculate the number of days.
data311_df["seconds_to_close"] = data311_df["time_to_close"].astype('timedelta64[s]')
data311_df["days_to_close"] = data311_df["seconds_to_close"]/60/60/24
 
# data311_df[["create_date_time", "date_closed_time", "time_to_close","seconds_to_close", "days_to_close"]]


data311_df[["create_date_time","due_date_time", "date_closed_time", "missed_due_date", "days_to_close"]]



,create_date_time,due_date_time,date_closed_time,missed_due_date,days_to_close
0,2017-01-01 00:30:00,2017-01-11 00:30:00,2017-02-02 13:20:00,1,32.534722
1,2017-01-01 05:22:00,2017-01-02 05:22:00,2017-01-04 11:00:00,1,3.234722
2,2017-01-01 05:23:00,2017-01-03 05:23:00,2017-01-04 03:40:00,1,2.928472
3,2017-01-01 06:43:00,2017-01-03 06:43:00,2017-01-01 09:10:00,0,0.102083
4,2017-01-01 08:45:00,2017-01-11 08:45:00,2017-01-01 11:20:00,0,0.107639
5,2017-01-01 09:49:00,2017-01-02 09:49:00,2017-01-01 12:00:00,0,0.090972
6,2017-01-01 10:23:00,2017-01-11 10:23:00,2017-01-01 12:55:00,0,0.105556
7,2017-01-01 10:30:00,2017-01-02 10:30:00,2017-01-12 02:50:00,1,10.680556
8,2017-01-01 10:56:00,2017-01-02 10:56:00,2017-01-02 16:10:00,1,1.218056
9,2017-01-01 11:04:00,2017-01-02 11:04:00,2017-01-04 03:35:00,1,2.688194


In [106]:
# bring in the weather data

weatherdata_df = pd.read_csv("../static/data/Final_selected_weather_data.csv")

print(weatherdata_df.shape)
weatherdata_df.head()

# checking the date lenght to make sure it matches with the 311 date length

# weatherdata_df["date_len"] = weatherdata_df["date_field"].str.len() 
# print(weatherdata_df[["date_field", "date_len"]])


(1180, 6)


,Unnamed: 0,date_field,tempMax,tempAvg,tempMin,precipitation
0,0,2017-01-01,74,67.5,64,0.30
1,1,2017-01-02,74,66.9,53,0.07
2,2,2017-01-03,75,61.9,49,1.22
3,3,2017-01-04,52,46.5,42,0.00
4,4,2017-01-05,56,50.4,44,0.00


In [107]:
# bring in the census data

censusdata_df = pd.read_csv("../Clean Data Files/census_data.csv")
censusdata_df.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Total Households,Total Owner Occupied,% Owner Occupied
0,77002,12370.0,34.1,72306.0,34779.0,6.984640,4457.0,1213.0,27.215616
1,77003,9646.0,34.1,59575.0,37760.0,27.628032,9597.0,4731.0,49.296655
2,77004,37642.0,28.3,48592.0,31067.0,19.733277,28125.0,9997.0,35.544889
3,77005,28233.0,36.5,180758.0,100896.0,3.715510,25438.0,20433.0,80.324711
4,77006,21945.0,34.3,82878.0,68705.0,7.304625,21280.0,8945.0,42.034774


In [108]:
# merge weather data into the 311 data

data_all1_df = data311_df.merge(weatherdata_df, how="left", left_on="create_date", right_on="date_field")

print(data_all1_df.shape)
data_all1_df.head()

# print(data_all[data_all["date_field"] == "2017-01-04"])

(861, 46)


,index,case number,sr location,county,district,neighborhood,tax id,trash quad,recycle quad,trash day,...,date_closed_time,time_to_close,seconds_to_close,days_to_close,Unnamed: 0,date_field,tempMax,tempAvg,tempMin,precipitation
0,6,12091836-101002444730,"3303 SAGE, HOUSTON TX 77056",HARRIS,G,GREATER UPTOWN,4.514000e+11,NaN,NaN,NaN,...,2017-02-02 13:20:00,32 days 12:50:00,2811000.0,32.534722,0,2017-01-01,74,67.5,64,0.3
1,12,12091839-101002444736,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,7.102100e+11,NE,NW,MONDAY,...,2017-01-04 11:00:00,3 days 05:38:00,279480.0,3.234722,0,2017-01-01,74,67.5,64,0.3
2,13,12091840-101002444737,"7701 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,4.200500e+11,NE,NW,MONDAY,...,2017-01-04 03:40:00,2 days 22:17:00,253020.0,2.928472,0,2017-01-01,74,67.5,64,0.3
3,15,12091841-101002444739,"7612 APPLETON, HOUSTON TX 77022",HARRIS,H,NORTHSIDE/NORTHLINE,7.102100e+11,NE,NW,MONDAY,...,2017-01-01 09:10:00,0 days 02:27:00,8820.0,0.102083,0,2017-01-01,74,67.5,64,0.3
4,23,12091843-101002444752,"7335 IWO JIMA, HOUSTON TX 77033",HARRIS,D,SOUTH PARK,7.518900e+11,SE,SE,FRIDAY,...,2017-01-01 11:20:00,0 days 02:35:00,9300.0,0.107639,0,2017-01-01,74,67.5,64,0.3


In [143]:
data_final_df = data_all1_df.merge(censusdata_df, how="left", left_on="zipcode", right_on="Zipcode") 

print(data_final_df.shape)

data_final_df = data_final_df.drop(columns=["Zipcode","date_field"])

print(data_final_df.columns.values)
# data_final_df.head()

data_final_df.describe()

data_final_df.isna().sum()

print(data_final_df[data_final_df["district"].isna()])

(861, 55)
['index' 'case number' 'sr location' 'county' 'district' 'neighborhood'
 'tax id' 'trash quad' 'recycle quad' 'trash day' 'heavy trash day'
 'recycle day' 'key map' 'management district' 'department' 'division'
 'sr type' 'queue' 'sla' 'status' 'sr create date' 'due date'
 'date closed' 'overdue' 'title' 'x' 'y' 'latitude' 'longitude'
 'channel type' 'zipcode' 'openclosetime' 'openduetime' 'year'
 'create_date' 'create_date_time' 'date_closed_time' 'time_to_close'
 'seconds_to_close' 'days_to_close' 'Unnamed: 0' 'tempMax' 'tempAvg'
 'tempMin' 'precipitation' 'Population' 'Median Age' 'Household Income'
 'Per Capita Income' 'Poverty Rate' 'Total Households'
 'Total Owner Occupied' '% Owner Occupied']
     index            case number                                 sr location  \
55     154  12091899-101002444887  5686 GRAND FLORAL, UNINCORPORATED TX 77041   
263    735            1.01002E+11              1302 MCDUGALD, HUMBLE TX 77338   
376   1009  12092234-101002445804    1

In [139]:
y=data_final_df["days_to_close"]

X_var=data_final_df[["district","sr type","tempMax","tempAvg","tempMin","precipitation","Population","Median Age",
                    "Household Income","Poverty Rate","% Owner Occupied"]]
X_var.isna().sum()
 

district            4
sr type             0
tempMax             0
tempAvg             0
tempMin             0
precipitation       0
Population          0
Median Age          0
Household Income    0
Poverty Rate        0
% Owner Occupied    0
dtype: int64